# Predict real estate prices w linear regression
03-predict-real-estate-prices-w-linear-regression-in-python

In [181]:
# Set-up libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [182]:
# Read data into dataframe
df = pd.read_csv('../00-Datasets/nyc-rolling-sales.csv')

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [184]:
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [185]:
# Check for missing values
df.isna().sum()
# none caught here but notice index 1 and 2 'SALE PRICE' values in line [7] of our code

Unnamed: 0                        0
BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING CLASS CATEGORY           0
TAX CLASS AT PRESENT              0
BLOCK                             0
LOT                               0
EASE-MENT                         0
BUILDING CLASS AT PRESENT         0
ADDRESS                           0
APARTMENT NUMBER                  0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
SALE DATE                         0
dtype: int64

In [186]:
# Check for duplicates
df.duplicated().sum()

0

In [187]:
# Explore tabular summary
df.describe()

,Unnamed: 0,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000
mean,10344.359878,2.998758,4237.218976,376.224015,10731.991614,2.025264,0.193559,2.249184,1789.322976,1.657485
std,7151.779436,1.289790,3568.263407,658.136814,1290.879147,16.721037,8.713183,18.972584,537.344993,0.819341
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4231.000000,2.000000,1322.750000,22.000000,10305.000000,0.000000,0.000000,1.000000,1920.000000,1.000000
50%,8942.000000,3.000000,3311.000000,50.000000,11209.000000,1.000000,0.000000,1.000000,1940.000000,2.000000
75%,15987.250000,4.000000,6281.000000,1001.000000,11357.000000,2.000000,0.000000,2.000000,1965.000000,2.000000
max,26739.000000,5.000000,16322.000000,9106.000000,11694.000000,1844.000000,2261.000000,2261.000000,2017.000000,4.000000


In [188]:
# Transform categorical features to numeric
df['SALE PRICE'] = pd.to_numeric(df['SALE PRICE'], errors='coerce')
df['GROSS SQUARE FEET'] = pd.to_numeric(df['GROSS SQUARE FEET'], errors='coerce')

In [189]:
# Replace missing values in 'SALE PRICE', 'GROSS SQUARE FEET' columns with mean
mean_square_feet = df['GROSS SQUARE FEET'].mean()
df['GROSS SQUARE FEET'].fillna(mean_square_feet, inplace=True)
mean_sale_price = df['SALE PRICE'].mean()
df['SALE PRICE'].fillna(mean_sale_price, inplace=True)

In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      84548 non-null  int64  
 1   BOROUGH                         84548 non-null  int64  
 2   NEIGHBORHOOD                    84548 non-null  object 
 3   BUILDING CLASS CATEGORY         84548 non-null  object 
 4   TAX CLASS AT PRESENT            84548 non-null  object 
 5   BLOCK                           84548 non-null  int64  
 6   LOT                             84548 non-null  int64  
 7   EASE-MENT                       84548 non-null  object 
 8   BUILDING CLASS AT PRESENT       84548 non-null  object 
 9   ADDRESS                         84548 non-null  object 
 10  APARTMENT NUMBER                84548 non-null  object 
 11  ZIP CODE                        84548 non-null  int64  
 12  RESIDENTIAL UNITS               

In [191]:
# Split the data into 80% train and 20% test
X = df['GROSS SQUARE FEET'].values.reshape(-1, 1)
y = df['SALE PRICE'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [192]:
# Build the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [193]:
# Feed the test data into the model
y_predict = model.predict(X_test)

actual_vs_predict = pd.DataFrame({'Actual':y_test.flatten(), 
                                'Prediction':y_predict.flatten()})
actual_vs_predict.sample(12)

,Actual,Prediction
5492,0.000000e+00,1.187372e+06
10481,4.800340e+05,1.300200e+06
11595,1.276456e+06,1.300200e+06
9063,1.527375e+06,7.155144e+05
10209,8.550000e+05,1.105718e+06
3433,3.150000e+05,8.727522e+05
8101,1.276456e+06,1.300200e+06
9268,1.575000e+05,7.155144e+05
12186,4.820000e+05,8.822905e+05
8519,1.470000e+06,1.178701e+06


In [194]:
# Evaluate model
model.score(X_test, y_test)

-0.5776661173794548

In [195]:
# Try to use more than one predictor variable
df['LAND SQUARE FEET'] = pd.to_numeric(df['LAND SQUARE FEET'], errors='coerce')
mean_square_feet = df['LAND SQUARE FEET'].mean()
df['LAND SQUARE FEET'].fillna(mean_square_feet, inplace=True)

X = pd.DataFrame(df[['LAND SQUARE FEET', 'GROSS SQUARE FEET']])
y = df['SALE PRICE'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

-0.9571329675734752

## Notes
The choices and assumptions made in this work:
* Not the best score. Needs a second look.
* Missing values found in important features 'SALE PRICE', 'GROSS SQUARE FEET', 'LAND SQUARE FEET'. Handled by filling in with mean. Also transformed from data type string to int.
* The dataset was split into 80% train and 20% test sets. No validation set was created or used. No hyperparameter tuning occurred.
* There are two model iterations. 
* First model iteration: Assumed 'GROSS SQUARE FEET' feature is a good predictor of the 'SALE PRICE' feature. These features were assigned accordingly to variables X and y, predictor and target. Evaluation metric showed negative score.
* Second model iteration: Assumed 'SALE PRICE', 'GROSS SQUARE FEET' were good predictors of the 'SALE PRICE' feature.  Evaluation metric showed lower negative score.
* Model was evaluated using R-squared or coefficient determination metric only. The best possible score is 1.0 and the worst could drop below zero. 